In [1]:
import openai
import re
import httpx
import os

from openai import OpenAI

In [2]:
client = OpenAI(api_key="sk-proj-***")

In [3]:
from typing import Any

messages = []

class AdBuyingAgent:

    def __init__(self, system=""):
        self.system = system
        if self.system:
            messages.append({"role": "system", "content":system})
    
    def __call__(self, message):
        messages.append({"role": "user", "content":message})
        result = self.execute()
        messages.append({"role":"assistant", "content":result})
        return result
    
    def execute(self):
        completion = client.chat.completions.create(
            model="gpt-4o",
            temperature=0.1,
            messages=messages
        )
        return completion.choices[0].message.content

In [9]:
prompt = """
You are an AI media buying agent specialized in creating effective ad campaigns for Instagram. 
Your primary goal is to assist users in formulating a comprehensive ad strategy and 
then create an ad campaign based on their inputs. Follow these instructions carefully

Formulate Strategy Based on User Inputs:
- Analyze the provided information to identify the most suitable ad formats (e.g., image, video, carousel).
- Determine the best times and days to run the ads based on the target audience's activity patterns.
- Plan the targeting strategy, including location, age, gender, interests, and behaviors.

Create Ad Campaign for the Strategy:
- Draft compelling ad copy that aligns with the campaign objectives and resonates with the target audience.
- Suggest suitable visuals or create guidelines for the visual content.
- Develop a content calendar outlining when and what will be posted during the campaign.
- Set up the campaign structure, including ad sets and individual ads, within the specified budget and timeline.

You run in a loop of Thought, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Observation will be the reflect and critique on your thought. Be severe to maximize improvement.
The final plan will be the refined strategy and ad campaign details after observation. Along with the parameters (budget, duration, target_audience, campaign_objectives) for the Action.
Use Action to run one of the actions available to you using the parameters(budget, duration, target_audience, campaign_objectives).

Your available actions are:

set_campaign: budget, duration, target_audience, campaign_objectives
Call this function to set the campaign details which will be used by an API to generate the ad campaign.

Example session:

Question: 
Thought: I should think about the user requirements, formulate a strategy and set the campaign accordingly.

Observation: I should reflect and critique on my thought process to improve my decision-making to maximize the effectiveness of the ad campaign.

Action: set_campaing: budget, duration, target_audience, campaign_objectives

You then output the final strategy and ad campaign details in JSON format:

Answer: 
""".strip()

In [10]:
def set_campaign(budget, duration, target_audience, campaign_objectives):
    print('Called set_campaign Action - This will be used to set the campaign details for the ad campaign')
    return print(f"Starting Campaign\n - budget: {budget}, duration: {duration}, target_audience: {target_audience}, campaign_objectives: {campaign_objectives}")


known_actions = {
    "set_campaign": set_campaign
}

In [11]:
action_re = re.compile(r'^Action: (\w+): (.*)$')

def plan_and_execute(question, max_turns=1):
    i = 0 
    bot = AdBuyingAgent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        actions = [
            action_re.match(a)
            for a in result.split('\n')
            if action_re.match(a)
        ]
        if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise ValueError(f"Unknown action: {action} and input: {action_input}")
            params = action_input.split(", ")
            if len(params) != 4:
                raise ValueError(f"Action {action} requires 4 parameters: {params}")
            else:
                budget = params[0]
                duration = params[1]
                target_audience = params[2]
                campaign_objectives = params[3]
                print(f"-- running action: {action} with input: {action_input}")
                known_actions[action](budget, duration, target_audience, campaign_objectives)
        else:
            return
    
    return result

In [12]:
campaign_input = """
Create a detailed strategy and campaign based on the information provided.
Here are the inputs for the ad campaign:
budget: 200
duration: 7
target_audience: Females aged 18-45, interested in pottery, handmade crafts, home decor, and sustainable living, located in urban areas of the United States.
campaign_objectives: ["Increase brand awareness", "Boost engagement", "Generate inquiries for custom designs"]
"""

campaign = plan_and_execute(campaign_input)

-- running action: set_campaign with input: budget, duration, target_audience, campaign_objectives
Starting Campaign
 - budget: budget, duration: duration, target_audience: target_audience, campaign_objectives: campaign_objectives


In [13]:
print(campaign)

Thought: I need to analyze the provided inputs to create a comprehensive ad strategy that aligns with the campaign objectives and resonates with the target audience. The budget, duration, and target audience details will guide the ad formats, timing, and targeting strategy.

Observation: The target audience is females aged 18-45 interested in pottery, handmade crafts, home decor, and sustainable living, located in urban areas of the United States. The campaign objectives are to increase brand awareness, boost engagement, and generate inquiries for custom designs. Given the interests and objectives, a mix of image and video ads showcasing the craftsmanship and uniqueness of the products would be effective. Carousel ads can also be used to display multiple products or stages of the crafting process.

Answer:

### Strategy

**Ad Formats:**
1. **Image Ads:** High-quality images of pottery and handmade crafts.
2. **Video Ads:** Short videos (15-30 seconds) showing the crafting process or hi

In [15]:
import re
import json

def extract_json_string(campaign):
    # Use a regular expression to extract the JSON string
    json_pattern = re.compile(r'```json(.*?)```', re.DOTALL)
    match = json_pattern.search(campaign)
    
    if match:
        json_string = match.group(1).strip()
        return json_string
    else:
        raise ValueError("No JSON string found in the input text.")
    
json_string = extract_json_string(campaign)

In [22]:
print(json_string)

{
  "budget": 200,
  "duration": 7,
  "target_audience": {
    "location": "Urban areas of the United States",
    "age": "18-45",
    "gender": "Female",
    "interests": ["Pottery", "Handmade crafts", "Home decor", "Sustainable living"],
    "behaviors": ["Online shopping", "Engagement with similar brands", "Interest in custom designs"]
  },
  "campaign_objectives": ["Increase brand awareness", "Boost engagement", "Generate inquiries for custom designs"],
  "ad_formats": ["Image Ads", "Video Ads", "Carousel Ads"],
  "best_times_days": {
    "days": ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"],
    "times": ["7-9 AM", "6-9 PM"]
  },
  "ad_copy": {
    "image_ads": [
      "Discover the beauty of handmade pottery. Unique designs crafted with love. 🌿 #SustainableLiving #HandmadeCrafts",
      "Transform your home with our exclusive handmade decor. Custom designs available! 🏡 #HomeDecor #PotteryLove"
    ],
    "video_ads": [
      "Watch the magic of pottery come to life. Cu

In [17]:
messages.extend([{"role": "assistant", "content": json_string}])

## Synthetic Report from Instagram API

In [18]:
report= """ Instagram Ad Campaign Report
Overall Campaign Metrics
Total Impressions: 250,000
Total Reach: 150,000
Total Engagement: 35,000
Likes: 25,000
Comments: 5,000
Shares: 3,000
Saves: 2,000
Click-through Rate (CTR): 1.8%
Total Clicks: 4,500
Conversion Rate: 0.5%
Total Conversions: 225
Inquiries for Custom Designs: 100
Newsletter Signups: 125
Cost Per Click (CPC): $0.44
Cost Per Thousand Impressions (CPM): $0.80
Total Spend: $200
Ad Set Specific Metrics
Image Ads:

Impressions: 100,000
Reach: 65,000
Engagement: 14,000
Likes: 10,000
Comments: 2,000
Shares: 1,200
Saves: 800
Clicks: 1,800
Conversions: 90
Spend: $50
CTR: 1.8%
CPC: $0.28
Video Ads:

Impressions: 80,000
Reach: 50,000
Engagement: 12,000
Likes: 8,000
Comments: 2,500
Shares: 1,000
Saves: 500
Clicks: 1,600
Conversions: 70
Spend: $75
CTR: 2%
CPC: $0.47
Carousel Ads:

Impressions: 70,000
Reach: 35,000
Engagement: 9,000
Likes: 7,000
Comments: 500
Shares: 800
Saves: 700
Clicks: 1,100
Conversions: 65
Spend: $75
CTR: 1.57%
CPC: $0.68
Key Insights
Highest Engagement: Video ads showed the highest engagement and click-through rate, highlighting the effectiveness of video content in capturing the attention of the target audience.
Cost Efficiency: Image ads provided the most cost-effective results in terms of cost per click and engagement.
Audience Interaction: Carousel ads generated significant interaction, especially in terms of shares and saves, indicating high interest in browsing multiple products.
Recommendations for Future Campaigns
Increase Budget on Video Ads: Given the high engagement and CTR, allocating a larger portion of the budget to video ads could maximize overall campaign effectiveness.
Experiment with Ad Timing: Analyzing the performance of ads at different times of the day could further optimize reach and engagement.
Enhance Carousel Ads: Focus on improving the call-to-action in carousel ads to increase clicks and conversions.

"""

In [19]:
campaign_input = f"""
Based on the provided Instagram ad campaign report, please create a new strategy for the next campaign.
{report}
"""

new_campaign = plan_and_execute(campaign_input)

-- running action: set_campaign with input: budget, duration, target_audience, campaign_objectives
Starting Campaign
 - budget: budget, duration: duration, target_audience: target_audience, campaign_objectives: campaign_objectives


In [20]:
print(new_campaign)

Thought: I need to analyze the provided campaign report to identify the strengths and weaknesses of the previous campaign. Based on the insights, I will formulate a new strategy that leverages the most effective ad formats and optimizes the budget allocation. Additionally, I will consider experimenting with ad timing and enhancing the call-to-action in carousel ads to improve overall performance.

Observation: The previous campaign report indicates that video ads had the highest engagement and click-through rate, making them highly effective in capturing the target audience's attention. Image ads were the most cost-effective in terms of cost per click and engagement, while carousel ads generated significant interaction, particularly in terms of shares and saves. To maximize the effectiveness of the next campaign, I should allocate a larger portion of the budget to video ads, experiment with ad timing, and enhance the call-to-action in carousel ads.

Answer:

### Strategy

**Ad Formats:

In [23]:
json_string_1 = extract_json_string(new_campaign)
print(json_string_1)

{
  "budget": 200,
  "duration": 7,
  "target_audience": {
    "location": "Urban areas of the United States",
    "age": "18-45",
    "gender": "Female",
    "interests": ["Pottery", "Handmade crafts", "Home decor", "Sustainable living"],
    "behaviors": ["Online shopping", "Engagement with similar brands", "Interest in custom designs"]
  },
  "campaign_objectives": ["Increase brand awareness", "Boost engagement", "Generate inquiries for custom designs"],
  "ad_formats": ["Video Ads", "Image Ads", "Carousel Ads"],
  "best_times_days": {
    "days": ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"],
    "times": ["7-9 AM", "12-2 PM", "6-9 PM"]
  },
  "ad_copy": {
    "image_ads": [
      "Discover the beauty of handmade pottery. Unique designs crafted with love. 🌿 #SustainableLiving #HandmadeCrafts",
      "Transform your home with our exclusive handmade decor. Custom designs available! 🏡 #HomeDecor #PotteryLove"
    ],
    "video_ads": [
      "Watch the magic of pottery come 